In [1]:
!pip install simpy

In [2]:
import simpy
import random
import statistics

In [3]:
!pip install Airport

  Preparing metadata (setup.py) ... done
  Created wheel for Airport: filename=airport-0.0.1-py3-none-any.whl size=1002 sha256=05569eda2e3d614d9cd54e2b46720fc421f2cec6c7470bc0319cf0717a2186a1
  Stored in directory: /root/.cache/pip/wheels/b4/9a/af/07c7950ad2809a07dc008bf3cb1fb4bfe4be91b597addeaf27
Successfully built Airport


In [11]:
wait_times = []
airport_total_times = []
check_total_times = []

class Airport(object):
    def __init__(self, env, num_checkers, num_personal_queue):
        self.env = env
        self.id_queue = simpy.Resource(env, num_checkers)
        self.personal_queue = [simpy.Resource(env, 1) for _ in range(num_personal_queue)]

    def id_queue_up(self, passenger):
        yield self.env.timeout(random.expovariate(1.0 / 0.75))

    def personal_queue_up(self, passenger):
        yield self.env.timeout(random.uniform(0.5, 1.0))

def enter_airport(env, passenger, airport):
    arrive_time = env.now

    with airport.id_queue.request() as request:
        yield request
        start_id_queue_time = env.now
        yield env.process(airport.id_queue_up(passenger))
        end_id_queue_time = env.now
        id_queue_time = end_id_queue_time - start_id_queue_time

    personal_queue_lens = [len(queue.queue) for queue in airport.personal_queue]
    shortest_queue_index = personal_queue_lens.index(min(personal_queue_lens))

    with airport.personal_queue[shortest_queue_index].request() as request:
        yield request
        start_personal_queue_time = env.now
        yield env.process(airport.personal_queue_up(passenger))
        end_personal_queue_time = env.now
        personal_queue_time = end_personal_queue_time - start_personal_queue_time
        queue_total_time = id_queue_time + personal_queue_time
        check_total_times.append(queue_total_time)

    leave_time = env.now
    total_time = leave_time - arrive_time
    airport_total_times.append(total_time)
    current_waiting_time = total_time - id_queue_time - personal_queue_time
    wait_times.append(current_waiting_time)

def run_airport(env, num_checkers, num_personal_queue):
    airport = Airport(env, num_checkers, num_personal_queue)
    for passenger in range(50):
        env.process(enter_airport(env, passenger, airport))

    while True:
        yield env.timeout(random.expovariate(50))
        passenger += 1
        env.process(enter_airport(env, passenger, airport))

def calc_wait_times(in_wait_times, in_check_times, in_total_times):
    average_wait_times = statistics.mean(in_wait_times)
    average_check_times = statistics.mean(in_check_times)
    average_airport_times = statistics.mean(in_total_times)
    return average_wait_times, average_check_times, average_airport_times

In [22]:
def main(num_checkers, num_personal_queue, num_reps):
  for rep in range(num_reps):
    random.seed(99)
    env = simpy.Environment()
    env.process(run_airport(env, num_checkers, num_personal_queue))
    env.run(until = 60 * 8)

    average_time_min, avg_c_time, avg_t_time = calc_wait_times(wait_times, check_total_times, airport_total_times)
    print(f"for {num_checkers} ID checkers, {num_personal_queue} personal queues and {num_reps} repetitions: ")
    print(f"===============================================")
    print(f"avg_total_time = {avg_t_time:.3f}\n"
          f"avg_check_time (both ID check and personal check) = {avg_c_time:.3f}\n"
          f"avg_wait_time = {average_time_min:.3f}\n")
    wait_times.clear()
    airport_total_times.clear()
    check_total_times.clear()

    return num_checkers, average_time_min

results_list = []
results_list.append(main(22, 22, 30))

for 22 ID checkers, 22 personal queues and 30 repetitions: 
avg_total_time = 106.493
avg_check_time (both ID check and personal check) = 1.494
avg_wait_time = 104.999



In [23]:
results_list.append(main(18, 18, 30))

for 18 ID checkers, 18 personal queues and 30 repetitions: 
avg_total_time = 131.734
avg_check_time (both ID check and personal check) = 1.499
avg_wait_time = 130.235



In [24]:
results_list.append(main(25, 25, 30))

for 25 ID checkers, 25 personal queues and 30 repetitions: 
avg_total_time = 86.969
avg_check_time (both ID check and personal check) = 1.502
avg_wait_time = 85.467



In [25]:
results_list.append(main(28, 28, 30))

for 28 ID checkers, 28 personal queues and 30 repetitions: 
avg_total_time = 62.790
avg_check_time (both ID check and personal check) = 1.492
avg_wait_time = 61.298



In [26]:
results_list.append(main(31, 31, 30))

for 31 ID checkers, 31 personal queues and 30 repetitions: 
avg_total_time = 45.293
avg_check_time (both ID check and personal check) = 1.493
avg_wait_time = 43.800



In [27]:
results_list.append(main(34, 34, 30))

for 34 ID checkers, 34 personal queues and 30 repetitions: 
avg_total_time = 23.741
avg_check_time (both ID check and personal check) = 1.490
avg_wait_time = 22.251



In [28]:
import plotly.express as px
import pandas as pd

In [29]:
results_df = pd.DataFrame(results_list, columns=['num_checkers', 'avg_wait_time'])

In [30]:
results_df

,num_checkers,avg_wait_time
0,22,104.999158
1,18,130.235406
2,25,85.467217
3,28,61.298480
4,31,43.800175
5,34,22.250906
